In [ ]:
import dai
import pandas as pd
import numpy as np
import time
from datetime import timedelta

In [ ]:
ed = dai.query("SELECT DISTINCT date FROM cn_stock_bar1d WHERE date >= '2024-01-01'").df()['date'].max().strftime("%Y-%m-%d")
sd = (pd.Timestamp(ed) - timedelta(days=45)).strftime("%Y-%m-%d")

In [ ]:
sql = """

WITH 

data1 AS (
    SELECT 
        *,
        make_date(year(date), month(date),  day(date))    AS day,
        make_time(hour(date), minute(date), second(date)) AS time,
        m_nanavg(volume, 5) / m_nanavg(volume, 30)        AS ATV,
        c_pct_rank(ATV)                                   AS rank_ATV,
    FROM cn_stock_bar1m
    WHERE instrument NOT LIKE '%BJ'
    ORDER BY date, instrument
),

data2 AS (
    SELECT
        *,
        day,
        time,
        AVG(rank_ATV) OVER(PARTITION BY day, instrument)      AS mean_rank_ATV,
        STDDEV(rank_ATV) OVER(PARTITION BY day, instrument)   AS std_rank_ATV,
        KURTOSIS(rank_ATV) OVER(PARTITION BY day, instrument) AS kurtosis_rank_ATV,
    FROM data1
    ORDER BY instrument, date
),

data3 AS (
    SELECT DISTINCT
        day AS date,
        instrument,
        mean_rank_ATV,
        std_rank_ATV,
        kurtosis_rank_ATV,
    FROM data2
),

data4 AS (
    SELECT
        date,
        instrument,
        mean_rank_ATV                                    AS alpha_hf_00010001,
        std_rank_ATV                                     AS alpha_hf_00010002,
        kurtosis_rank_ATV                                AS alpha_hf_00010003,
        mean_rank_ATV / std_rank_ATV + kurtosis_rank_ATV AS alpha_hf_00010004,
    FROM data3
)

SELECT * 
FROM  data4
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument

"""